## Script Oil Production

### Import de Bibliotecas

In [32]:
#Automação WEB
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from pyautogui import alert, confirm

#Mover/Carregar Arquivos
import os
from pathlib import Path
import shutil
from zipfile import ZipFile
import pandas as pd

#Conexão ao DB
import pyodbc

#Envio de E-mail
import smtplib
import email.message

### Web Scraping

In [33]:
confirm(text='Script de atualização do Dashboard.', title='Script', buttons=['Confirmar', 'Cancelar'])

chromedriver_autoinstaller.install()
Navegador = webdriver.Chrome()

Link = "https://www.kaggle.com/datasets/sazidthe1/oil-production"
Navegador.get(Link)
sleep(1)

Button_Sign_In = Navegador.find_element('xpath', '//*[@id="site-container"]/div/div[4]/div[2]/div[2]/div/div[1]/a/button')
Button_Sign_In.click()
sleep(1)

Button_Email = Navegador.find_element('xpath', '//*[@id="site-content"]/div[2]/div/div/div[1]/form/div/div/div[1]/button[2]')
Button_Email.click()
sleep(1)

Email_Address = Navegador.find_element('xpath', '//*[@id=":r7:"]')
Email_Address.send_keys('emiliano.junqueira@gmail.com')

Password = Navegador.find_element('xpath', '//*[@id=":r8:"]')
Password .send_keys('K@ggle0051')

Sign_In = Navegador.find_element('xpath', '//*[@id="site-content"]/div[2]/div[1]/div/div[1]/form/div/div[4]/button[2]')
Sign_In.click()
sleep(5)

Download = Navegador.find_element('xpath', '//*[@id="site-content"]/div[2]/div/div[2]/div/div[2]/div[1]/div/div[2]/div[1]/button[1]')
Download.click()
sleep(5)


Navegador.quit()

### Mover e Descompactar o Arquivo


In [34]:
Origin = ('C:/Users/Miza/Downloads/archive.zip')
Destiny = Path('C:/Users/Miza/Desktop/CoderHouse/Data_Analytics')

shutil.copy2(Origin, Destiny)

if os.path.exists(Origin):
    os.remove(Origin)

with ZipFile("archive.zip", "r") as zip:
    zip.extractall()

### Conexão ao DB

In [35]:
dados_conexion = (
	"Driver={SQL Server};"
	"Server=DESKTOP-2TU1ACJ;"
	"Database=Oil_Production;"
	)
conexao = pyodbc.connect(dados_conexion)
cursor = conexao.cursor()

### Carregar/Analisar os Dados 

In [36]:
df = pd.read_csv("C://Users//Miza//Desktop//CoderHouse//Data_Analytics//oil_production_statistics.csv", delimiter=',', encoding='unicode_escape',)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C://Users//Miza//Desktop//Data_Analytics//CoderHouse//oil_production_statistics.csv'

In [ ]:
df.info()

### Delete e Load dos Dados ao DB

In [ ]:
cursor.execute('TRUNCATE TABLE oil_production_statistics')
cursor.commit()

In [ ]:
for index, row in df.iterrows():
    sql = "INSERT INTO oil_production_statistics (country_name, type, product, flow, year, value) VALUES (?, ?, ?, ?, ?, ?)"
    val = (row['country_name'], row['type'], row['product'], row['flow'], row['year'], row['value'])
    cursor.execute(sql, val)
    conexao.commit()

conexao.close()  

### Envio do Email de aviso da Atualização

In [ ]:
def sendEmail():
    body_email= """
    
    <p> Olá!! </p>
    <p> Segue o link do arquivo para visualização: <a href="https://app.powerbi.com/view?r=eyJrIjoiMDU3MDA0MmYtYzVhZS00MDE5LWJmZTEtZDdiZmVkZjQ0ODIzIiwidCI6IjljNDk2NGIzLWFjODEtNDE4OC04ZGFhLWNjZWQwODZlYjVhMSJ9"> 
    Oil Production. </a> </p>   

    <p> Att,, </p>  
    
    """
    
    msg = email.message.Message()
    msg['Subject'] = "Dashboard Atualizado"
    msg['From'] = "emiliano.junqueira@gmail.com"
    msg['To'] = "emiliano.junqueira@gmail.com"
    password = 'ygep xoth agie mdek'
    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(body_email)
    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    s.login(msg['From'], password)
    s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    s.quit()

sendEmail() 

alert('Script chegou ao Fim.')